In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
first_dataset = pd.read_csv('dataset.csv')
first_dataset.head(10)

In [ ]:
second_dataset = pd.read_csv('IPL Matches 2008-2020.csv')
second_dataset.head()

In [ ]:
total_runs = first_dataset.groupby(['match_id','inning']).sum()['total_runs'].add(1).reset_index()

In [ ]:
total_runs = total_runs[total_runs['inning']==1]

In [ ]:
total_runs

In [ ]:
match_df = second_dataset.merge(total_runs[['match_id','total_runs']],left_on='id',right_on='match_id')

In [ ]:
match_df

In [ ]:
match_df['team1'].value_counts()

In [ ]:
match_df['team1'] = match_df['team1'].replace('Delhi Daredevils','Delhi Capitals')
match_df['team2'] = match_df['team2'].replace('Delhi Daredevils','Delhi Capitals')

match_df['team1'] = match_df['team1'].replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].replace('Deccan Chargers','Sunrisers Hyderabad')


In [ ]:
teams = ['Kolkata Knight Riders', 
         'Chennai Super Kings',
         'Kings XI Punjab', 
         'Rajasthan Royals',
         'Mumbai Indians',
         'Delhi Capitals',
         'Royal Challengers Bangalore',
         'Sunrisers Hyderabad']

In [ ]:
match_df = match_df[match_df['team1'].isin(teams)]
match_df = match_df[match_df['team2'].isin(teams)]

In [ ]:
match_df['team1'].value_counts()

In [ ]:
match_df['team2'].value_counts()

In [ ]:
match_df.shape

In [ ]:
match_df

In [ ]:
match_df = match_df[['id','city','winner','total_runs']]

In [ ]:
match_df = match_df.rename(columns={'id':'match_id'})

In [ ]:
second_innings = match_df.merge(first_dataset,on='match_id')
second_innings

In [ ]:
second_innings = second_innings[second_innings['inning']==2]
second_innings.shape

In [ ]:
second_innings['batting_team'] = second_innings['batting_team'].replace('Delhi Daredevils','Delhi Capitals')
second_innings['bowling_team'] = second_innings['bowling_team'].replace('Delhi Daredevils','Delhi Capitals')

second_innings['batting_team'] = second_innings['batting_team'].replace('Deccan Chargers','Sunrisers Hyderabad')
second_innings['bowling_team'] = second_innings['bowling_team'].replace('Deccan Chargers','Sunrisers Hyderabad')

In [ ]:
second_innings['batting_team'].unique()

In [ ]:
second_innings['current_Score'] = second_innings.groupby('match_id').cumsum()['total_runs_y']
second_innings

In [ ]:
second_innings['runs_left'] = second_innings['total_runs_x'] - second_innings['current_Score']
second_innings

In [ ]:
second_innings['balls_left'] = 126 - (second_innings['over']*6 +second_innings['ball'])
second_innings

## Getting wickets

In [ ]:
second_innings['player_dismissed'] = second_innings['player_dismissed'].fillna("0")
second_innings['player_dismissed'] = second_innings['player_dismissed'].apply(lambda x:x if x=="0" else "1")
second_innings['player_dismissed'] = second_innings['player_dismissed'].astype('int')
wickets = second_innings.groupby('match_id').cumsum()['player_dismissed']
second_innings['wickets_remaining'] = 10-wickets
second_innings

### Current Run Rate

In [ ]:
second_innings['crr'] = round((second_innings['current_Score']*6)/(120 - second_innings['balls_left']),2)
second_innings

### Required Run Rate

In [ ]:
second_innings['rrr'] = round((second_innings['runs_left']*6)/(second_innings['balls_left']),2)
second_innings

### finding winner 

In [ ]:
def winner(team):
    return 1 if team['batting_team'] == team['winner'] else 0

In [ ]:
second_innings['results'] = second_innings.apply(winner,axis=1)
second_innings

### Only Taking required columns

In [ ]:
required_data = second_innings[['batting_team','bowling_team','city','runs_left','balls_left','wickets_remaining','total_runs_x','crr','rrr','results']]
required_data

### Sampling Data

In [3]:
final_data = required_data.sample(required_data.shape[0])

NameError: name 'required_data' is not defined

### Dropping NUll Values

In [ ]:
final_data.dropna(inplace=True)

### Removing rows where balls_left==0 to remove -inf and inf from rrr column

In [ ]:
final_data = final_data[final_data['balls_left']!=0]

### Storing data in new csv

In [ ]:
final_data.to_csv("final_data.csv", index=False)

### Building Model

In [ ]:
from sklearn.model_selection import train_test_split as tts

In [ ]:
x = final_data.iloc[:, :-1]
y = final_data.iloc[:,-1]

In [ ]:
x_train,x_test,y_train,y_test = tts(x,y,test_size=0.25,random_state=100)
x_train

In [ ]:
x_test

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
],
remainder='passthrough')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
])


In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
y_pred = pipe.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
pipe.predict_proba(x_test)[0]

In [ ]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

In [ ]:
final_data['city'].unique()

## Random Forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
],
remainder='passthrough')

pipe = Pipeline([
    ('step1',trf),
    ('step2',RandomForestClassifier())
])

pipe.fit(x_train,y_train)

y_pred = pipe.predict(x_test)

accuracy_score(y_test,y_pred)


In [ ]:
pipe.predict_proba(x_test)[0]

In [ ]:
pickle.dump(pipe,open('pipe2.pkl','wb'))